# Feature Analysis
Let's do a little feature analysis! Mostly I'm just going to be pulling the data from Spotify's APIs to get even more cool data from my primitive data. (Look at how far we've come! Just started out with a track name and artist name)

In [1]:
from os import path
import numpy as np
import pandas as pd
from IPython.display import display

if path.exists("data/processed/NewTracksData.csv"):
    tracks_data_df = pd.read_csv("data/processed/NewTracksData.csv")
    tracks_data_df.drop(tracks_data_df.columns[0], axis=1, inplace=True)
    print(tracks_data_df.shape)
    display(tracks_data_df.head())
else:
    print("Clean data from other journal before continuing")

/home/jcroderman/.pyenv/versions/3.7.5/lib/python3.7/site-packages/pandas/compat/__init__.py:120: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


(21161, 14)


,artist_name,track_name,ms_played,date,album_id,album_name,album_release_date,album_total_tracks,artist_names,duration_ms,explicit,id,name,popularity
0,Mingus Big Band,Haitian Fight Song,680,2020-01-01 00:51:00,0Gwu5X7W1mrkSTk2uZ25cv,Blues & Politics,1999-06-29,8,['Mingus Big Band'],499826,False,4hns23kYYZg0BhDwXeDxB1,Haitian Fight Song,31
1,Andy Martin & Vic Lewis,Everything You Is,3282,2020-01-01 00:52:00,7By1lfK4fTIs2YsMvA0FWH,The Project,2004-01-01,10,['Andy Martin & Vic Lewis'],459106,False,3sLmks6fCY40bBSGDjU4FO,Everything You Is,0
2,Guy Lombardo & His Royal Canadians,Auld Lang Syne,66171,2020-01-01 00:52:00,3f22Ap0VSZYWsqrGcphUnY,Christmas Classics,2004-01-01,16,['Guy Lombardo & His Royal Canadians'],128000,False,4Saza06xljloZwotqXdNle,Auld Lang Syne,31
3,Thad Jones,To You,1496,2020-01-01 00:52:00,5gfrrR8BnDgFhqGWcQaWFe,And the Danish Radio Big Band & Eclipse,2013-05-10,22,['Thad Jones'],255466,False,2408a07TNDga6lMlaIFLEU,To You,7
4,Count Basie,All Of Me,150773,2020-01-01 00:55:00,2kAN1sZjSQQDkusyXyngep,Frankly Basie / Count Basie Plays The Hits Of ...,1963-01-01,15,['Count Basie'],150773,False,0rupt7DuLo3WGecL3cyi19,All Of Me,15


In [2]:
combined_tracks_data_df = tracks_data_df[["id", "date", "ms_played", "album_id"]]
combined_tracks_data_df.head()

,id,date,ms_played,album_id
0,4hns23kYYZg0BhDwXeDxB1,2020-01-01 00:51:00,680,0Gwu5X7W1mrkSTk2uZ25cv
1,3sLmks6fCY40bBSGDjU4FO,2020-01-01 00:52:00,3282,7By1lfK4fTIs2YsMvA0FWH
2,4Saza06xljloZwotqXdNle,2020-01-01 00:52:00,66171,3f22Ap0VSZYWsqrGcphUnY
3,2408a07TNDga6lMlaIFLEU,2020-01-01 00:52:00,1496,5gfrrR8BnDgFhqGWcQaWFe
4,0rupt7DuLo3WGecL3cyi19,2020-01-01 00:55:00,150773,2kAN1sZjSQQDkusyXyngep


## Getting the Extra Data
Thankfully, Spotify is FANTASTIC with its documentation and API specs, as they are all quite consistent! (Thank you Spotify) So, this allows me to create one single method to pull all of the data from the different endpoints!

In [3]:
import requests
import re
import time
from urllib.parse import urlencode
token = open('token.txt', 'r').read().strip()

base_url = "https://api.spotify.com/v1/"

def get_spotify_data(ids, endpoint, sublist_length=50):
    key_name = endpoint.replace("-", "_")
    ids_data = []
    for i in range((len(ids) // sublist_length) + 1):
        sub_ids = ",".join(ids[i * sublist_length:(i + 1) * sublist_length])
        query_obj = {"ids": sub_ids}
        res = requests.get(base_url + endpoint + "?" + urlencode(query_obj), headers={
            "Authorization": "Bearer " + token
        })
        if res.status_code != 200:
            raise Exception("Spotify call failed, something has gone wrong")
        res_json = res.json()
        ids_data += res_json[key_name]
    return ids_data

First let's get the full track data (yeah I know we did this in the cleaning step, but that was mostly just to get the track `id`s. This time, we're doing **feature engineering**! It's only proper to do it again here and pretend like I never did that in the first place).

We'll start off by gathering the data, then we'll clean it up later:

In [4]:
track_ids = list(combined_tracks_data_df["id"].drop_duplicates())

if not path.exists("data/raw/FullTrackDataRaw.csv"):
    full_track_data = get_spotify_data(track_ids, "tracks")
    full_track_data_df = pd.DataFrame({"id": track_ids, "data": full_track_data})    
    full_track_data_df.to_csv("data/raw/FullTrackDataRaw.csv")
else:
    full_track_data_df = pd.read_csv("data/raw/FullTrackDataRaw.csv")
    full_track_data_df.drop(full_track_data_df.columns[0], axis=1, inplace=True)

full_track_data_df.head()

,id,data
0,4hns23kYYZg0BhDwXeDxB1,"{'album': {'album_type': 'album', 'artists': [..."
1,3sLmks6fCY40bBSGDjU4FO,"{'album': {'album_type': 'album', 'artists': [..."
2,2408a07TNDga6lMlaIFLEU,"{'album': {'album_type': 'album', 'artists': [..."
3,4Saza06xljloZwotqXdNle,"{'album': {'album_type': 'compilation', 'artis..."
4,0rupt7DuLo3WGecL3cyi19,"{'album': {'album_type': 'album', 'artists': [..."


Track feature data, this will give us all sorts of neat metrics calculated by Spotify:

In [5]:
if not path.exists("data/raw/TrackFeatureDataRaw.csv"):
    track_feature_data = get_spotify_data(track_ids, "audio-features")
    track_feature_data_df = pd.DataFrame({"id": track_ids, "data": track_feature_data})
    track_feature_data_df.to_csv("data/raw/TrackFeatureDataRaw.csv")
else:
    track_feature_data_df = pd.read_csv("data/raw/TrackFeatureDataRaw.csv")
    track_feature_data_df.drop(track_feature_data_df.columns[0], axis=1, inplace=True)

track_feature_data_df.head()

,id,data
0,4hns23kYYZg0BhDwXeDxB1,"{'danceability': 0.386, 'energy': 0.392, 'key'..."
1,3sLmks6fCY40bBSGDjU4FO,"{'danceability': 0.581, 'energy': 0.517, 'key'..."
2,2408a07TNDga6lMlaIFLEU,"{'danceability': 0.138, 'energy': 0.11, 'key':..."
3,4Saza06xljloZwotqXdNle,"{'danceability': 0.24, 'energy': 0.245, 'key':..."
4,0rupt7DuLo3WGecL3cyi19,"{'danceability': 0.529, 'energy': 0.176, 'key'..."


Album data, to figure out more data about the albums I listen to:

In [6]:
album_ids = list(combined_tracks_data_df["album_id"].drop_duplicates())

if not path.exists("data/raw/AlbumDataRaw.csv"):
    album_data = get_spotify_data(album_ids, "albums", 20)
    album_data_df = pd.DataFrame({"id": album_ids, "data": album_data})
    album_data_df.to_csv("data/raw/AlbumDataRaw.csv")
else:
    album_data_df = pd.read_csv("data/raw/AlbumDataRaw.csv")
    album_data_df.drop(album_data_df.columns[0], axis=1, inplace=True)

album_data_df.head()

,id,data
0,0Gwu5X7W1mrkSTk2uZ25cv,"{'album_type': 'album', 'artists': [{'external..."
1,7By1lfK4fTIs2YsMvA0FWH,"{'album_type': 'album', 'artists': [{'external..."
2,5gfrrR8BnDgFhqGWcQaWFe,"{'album_type': 'album', 'artists': [{'external..."
3,3f22Ap0VSZYWsqrGcphUnY,"{'album_type': 'compilation', 'artists': [{'ex..."
4,2kAN1sZjSQQDkusyXyngep,"{'album_type': 'album', 'artists': [{'external..."


We will also do artist data later on, but I need the artists from albums first!

## Cleaning the Extra Data

Since we are getting the raw data from the API endpoints, we should clean it up by choosing only the pieces that we care about. These are the methods we'll use to help with the cleaning:

In [7]:
import ast # Gonna use ast because json has issues with single quotes
import json

# This will be used to show what keys are returned from the endpoint
def print_nested_keys(obj, offset=""):
    if obj is None:
        return
    if isinstance(obj, list):
        if len(obj) == 0:
            return
        obj = obj[0]
    keys = obj.keys()
    for key in keys:
        print(offset + key)
        try:
            print_nested_keys(obj[key], offset + "  ")
        except:
            pass

In [8]:
from datetime import date

def convert_release_date(date_str):
    if len(date_str) == 4:
        return date(int(date_str), 1, 1)
    elif len(date_str) == 7:
        return date(int(date_str[0:4]), int(date_str[5:7]), 1)
    else:
        return date(int(date_str[0:4]), int(date_str[5:7]), int(date_str[8:10]))

def get_data_values(entry, accessor_dict):
    data_str = entry["data"]
    data = None
    try:
        data = ast.literal_eval(data_str)
    except:
        try:
            data = json.loads(data_str)
        except:
            print("Invalid: ")
            print(data_str)
            pass
    finally:
        for new_entry_key, data_access_key in accessor_dict.items():
            if callable(data_access_key):
                entry[new_entry_key] = data_access_key(data)
            elif isinstance(data_access_key, list):
                for access in data_access_key:
                    entry[new_entry_key] = data[access] if not new_entry_key in entry else entry[new_entry_key][access]
            else:
                entry[new_entry_key] = data[data_access_key]
    return entry

In [9]:
full_track_data = full_track_data_df["data"].iloc[0]
print("Full Track Data:")
print_nested_keys(ast.literal_eval(full_track_data), "  ")

Full Track Data:
  album
    album_type
    artists
      external_urls
        spotify
      href
      id
      name
      type
      uri
    available_markets
    external_urls
      spotify
    href
    id
    images
      height
      url
      width
    name
    release_date
    release_date_precision
    total_tracks
    type
    uri
  artists
    external_urls
      spotify
    href
    id
    name
    type
    uri
  available_markets
  disc_number
  duration_ms
  explicit
  external_ids
    isrc
  external_urls
    spotify
  href
  id
  is_local
  is_playable
  name
  popularity
  preview_url
  track_number
  type
  uri


In [10]:
if not path.exists("data/ready/FullTrackData.csv"):
    full_track_data_accessor = {
        "album_id": ["album", "id"],
        "album_name": ["album", "name"],
        "album_release_date": lambda data: convert_release_date(data["album"]["release_date"]),
        "album_total_tracks": ["album", "total_tracks"],
        "artist_ids": lambda data: list(map(lambda artist_data: artist_data["id"], data["artists"])),
        "artist_names": lambda data: list(map(lambda artist_data: artist_data["name"], data["artists"])),
        "duration_ms": "duration_ms",
        "explicit": "explicit",
        "name": "name",
        "popularity": "popularity"
    }
    full_track_data_df = full_track_data_df.apply(lambda entry: get_data_values(entry, full_track_data_accessor), axis=1).drop(columns=["data"])
    full_track_data_df.to_csv("data/ready/FullTrackData.csv")
else:
    full_track_data_df = pd.read_csv("data/ready/FullTrackData.csv")
    full_track_data_df.drop(full_track_data_df.columns[0], axis=1, inplace=True)

full_track_data_df.head()

,id,album_id,album_name,album_release_date,album_total_tracks,artist_ids,artist_names,duration_ms,explicit,name,popularity,genres
0,4hns23kYYZg0BhDwXeDxB1,0Gwu5X7W1mrkSTk2uZ25cv,Blues & Politics,1999-06-29,8,['54YNxT02JdAApvFBhD8ea0'],['Mingus Big Band'],499826,False,Haitian Fight Song,30,"['bebop', 'big band', 'modern big band', 'jazz']"
1,3sLmks6fCY40bBSGDjU4FO,7By1lfK4fTIs2YsMvA0FWH,The Project,2004-01-01,10,['5zS6TsJ4lQFUGePSHAXaI9'],['Andy Martin & Vic Lewis'],459106,False,Everything You Is,0,[]
2,2408a07TNDga6lMlaIFLEU,5gfrrR8BnDgFhqGWcQaWFe,And the Danish Radio Big Band & Eclipse,2013-05-10,22,['6DbqS0X8cSFOPGsvyze2yh'],['Thad Jones'],255466,False,To You,7,"['hard bop', 'jazz trumpet', 'soul jazz', 'big..."
3,4Saza06xljloZwotqXdNle,3f22Ap0VSZYWsqrGcphUnY,Christmas Classics,2004-01-01,16,['5fJ4w85NxFXyWlPU9wH6BE'],['Guy Lombardo & His Royal Canadians'],128000,False,Auld Lang Syne,34,[]
4,0rupt7DuLo3WGecL3cyi19,2kAN1sZjSQQDkusyXyngep,Frankly Basie / Count Basie Plays The Hits Of ...,1963-01-01,15,['2jFZlvIea42ZvcCw4OeEdA'],['Count Basie'],150773,False,All Of Me,15,"['stride', 'lounge', 'swing', 'vocal jazz', 'b..."


In [11]:
track_feature_data = track_feature_data_df["data"].iloc[0]
print("Track Feature Data:")
print_nested_keys(ast.literal_eval(track_feature_data), "  ")

Track Feature Data:
  danceability
  energy
  key
  loudness
  mode
  speechiness
  acousticness
  instrumentalness
  liveness
  valence
  tempo
  type
  id
  uri
  track_href
  analysis_url
  duration_ms
  time_signature


In [12]:
if not path.exists("data/ready/TrackFeatureData.csv"):
    track_feature_data_accessor = {x: x for x in [
        "danceability",
        "energy",
        "key",
        "loudness",
        "mode",
        "speechiness",
        "acousticness",
        "instrumentalness",
        "liveness",
        "valence",
        "tempo",
        "time_signature",
    ]}
    track_feature_data_df = track_feature_data_df.apply(lambda entry: get_data_values(entry, track_feature_data_accessor), axis=1).drop(columns=["data"])
    track_feature_data_df.to_csv("data/ready/TrackFeatureData.csv")
else:
    track_feature_data_df = pd.read_csv("data/ready/TrackFeatureData.csv")
    track_feature_data_df.drop(track_feature_data_df.columns[0], axis=1, inplace=True)

track_feature_data_df.head()

,id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,boringness
0,4hns23kYYZg0BhDwXeDxB1,0.386,0.392,2,-14.123,0,0.0408,0.536,0.794,0.1670,0.3990,89.153,4,152.830
1,3sLmks6fCY40bBSGDjU4FO,0.581,0.517,5,-10.203,0,0.0378,0.901,0.794,0.1080,0.5530,138.751,4,238.348
2,2408a07TNDga6lMlaIFLEU,0.138,0.110,5,-15.402,1,0.0337,0.945,0.943,0.1110,0.0534,61.730,4,71.128
3,4Saza06xljloZwotqXdNle,0.240,0.245,5,-10.777,1,0.0327,0.979,0.788,0.0947,0.1120,83.922,3,121.645
4,0rupt7DuLo3WGecL3cyi19,0.529,0.176,7,-17.229,1,0.0398,0.762,0.720,0.0529,0.7910,150.108,4,203.379


In [13]:
album_data = album_data_df["data"].iloc[0]
print("Album Data:")
print_nested_keys(ast.literal_eval(album_data), "  ")

Album Data:
  album_type
  artists
    external_urls
      spotify
    href
    id
    name
    type
    uri
  available_markets
  copyrights
    text
    type
  external_ids
    upc
  external_urls
    spotify
  genres
  href
  id
  images
    height
    url
    width
  label
  name
  popularity
  release_date
  release_date_precision
  total_tracks
  tracks
    href
    items
      artists
        external_urls
          spotify
        href
        id
        name
        type
        uri
      available_markets
      disc_number
      duration_ms
      explicit
      external_urls
        spotify
      href
      id
      is_local
      name
      preview_url
      track_number
      type
      uri
    limit
    next
    offset
    previous
    total
  type
  uri


In [14]:
if not path.exists("data/ready/AlbumData.csv"):
    album_data_accessor = {
        "album_type": "album_type",
        "artist_ids": lambda data: list(map(lambda artist_data: artist_data["id"], data["artists"])),
        "artist_names": lambda data: list(map(lambda artist_data: artist_data["name"], data["artists"])),
        "copyright_texts": lambda data: list(map(lambda copyright: copyright["text"], data["copyrights"])),
        "genres": "genres",
        "image_url": lambda data: data["images"][0]["url"],
        "label": "label",
        "name": "name",
        "popularity": "popularity",
        "release_date": lambda data: convert_release_date(data["release_date"]),
        "total_tracks": "total_tracks"
    }
    album_data_df = album_data_df.apply(lambda entry: get_data_values(entry, album_data_accessor), axis=1).drop(columns=["data"])
    album_data_df.to_csv("data/ready/AlbumData.csv")
else:
    album_data_df = pd.read_csv("data/ready/AlbumData.csv")
    album_data_df.drop(album_data_df.columns[0], axis=1, inplace=True)

album_data_df.head()

,id,album_type,artist_ids,artist_names,copyright_texts,genres,image_url,label,name,popularity,release_date,total_tracks
0,0Gwu5X7W1mrkSTk2uZ25cv,album,['54YNxT02JdAApvFBhD8ea0'],['Mingus Big Band'],"['1999 Francis Dreyfus Music SARL, a BMG Compa...",[],https://i.scdn.co/image/ab67616d0000b27315338b...,Dreyfus Jazz,Blues & Politics,27,1999-06-29,8
1,7By1lfK4fTIs2YsMvA0FWH,album,['5zS6TsJ4lQFUGePSHAXaI9'],['Andy Martin & Vic Lewis'],"['2004 Drewbone Music', '2004 Drewbone Music']",[],https://i.scdn.co/image/ab67616d0000b273750e6c...,Drewbone Music,The Project,1,2004-01-01,10
2,5gfrrR8BnDgFhqGWcQaWFe,album,['6DbqS0X8cSFOPGsvyze2yh'],['Thad Jones'],['(C) 2013 Storyville Records'],[],https://i.scdn.co/image/ab67616d0000b2731a5d61...,Storyville,And the Danish Radio Big Band & Eclipse,16,2013-05-10,22
3,3f22Ap0VSZYWsqrGcphUnY,compilation,['0LyfQWJT6nXafLPZqxe9Of'],['Various Artists'],"['© 2004 Capitol Catalog', 'This Compilation ℗...",[],https://i.scdn.co/image/ab67616d0000b2733cfd1f...,Capitol Records,Christmas Classics,66,2004-01-01,16
4,2kAN1sZjSQQDkusyXyngep,album,['2jFZlvIea42ZvcCw4OeEdA'],['Count Basie'],"['© 1963 The Verve Music Group, a Division of ...",[],https://i.scdn.co/image/ab67616d0000b27364e3d6...,Verve,Frankly Basie / Count Basie Plays The Hits Of ...,17,1963-01-01,15


Now that we have the album data, let's get that artist data!

In [15]:
album_artist_ids = [artist_id for artist_ids in album_data_df["artist_ids"].values for artist_id in ast.literal_eval(artist_ids)]
track_artist_ids = [artist_id for artist_ids in full_track_data_df["artist_ids"].values for artist_id in ast.literal_eval(artist_ids)]
artist_ids = list(dict.fromkeys(album_artist_ids + track_artist_ids))

if not path.exists("data/raw/ArtistDataRaw.csv"):
    artist_data = get_spotify_data(artist_ids, "artists", 50)
    artist_data_df = pd.DataFrame({"id": artist_ids, "data": artist_data})
    artist_data_df.to_csv("data/raw/ArtistDataRaw.csv")
else:
    artist_data_df = pd.read_csv("data/raw/ArtistDataRaw.csv")
    artist_data_df.drop(artist_data_df.columns[0], axis=1, inplace=True)

artist_data_df.head()

,id,data
0,54YNxT02JdAApvFBhD8ea0,{'external_urls': {'spotify': 'https://open.sp...
1,5zS6TsJ4lQFUGePSHAXaI9,{'external_urls': {'spotify': 'https://open.sp...
2,6DbqS0X8cSFOPGsvyze2yh,{'external_urls': {'spotify': 'https://open.sp...
3,0LyfQWJT6nXafLPZqxe9Of,{'external_urls': {'spotify': 'https://open.sp...
4,2jFZlvIea42ZvcCw4OeEdA,{'external_urls': {'spotify': 'https://open.sp...


In [16]:
artist_data = artist_data_df["data"].iloc[0]
print("Artist Data:")
print_nested_keys(ast.literal_eval(artist_data), "  ")

Artist Data:
  external_urls
    spotify
  followers
    href
    total
  genres
  href
  id
  images
    height
    url
    width
  name
  popularity
  type
  uri


In [17]:
if not path.exists("data/ready/ArtistData.csv"):
    artist_data_accessor = {x: x for x in [
        "genres",
        "name",
        "popularity"
    ]}
    artist_data_df = artist_data_df.apply(lambda entry: get_data_values(entry, artist_data_accessor), axis=1).drop(columns=["data"])
    artist_data_df.to_csv("data/ready/ArtistData.csv")
else:
    artist_data_df = pd.read_csv("data/ready/ArtistData.csv")
    artist_data_df.drop(artist_data_df.columns[0], axis=1, inplace=True)

artist_data_df.head()

,id,genres,name,popularity
0,54YNxT02JdAApvFBhD8ea0,"['bebop', 'big band', 'jazz', 'modern big band']",Mingus Big Band,33
1,5zS6TsJ4lQFUGePSHAXaI9,[],Andy Martin & Vic Lewis,0
2,6DbqS0X8cSFOPGsvyze2yh,"['bebop', 'big band', 'contemporary post-bop',...",Thad Jones,41
3,0LyfQWJT6nXafLPZqxe9Of,[],Various Artists,0
4,2jFZlvIea42ZvcCw4OeEdA,"['adult standards', 'bebop', 'big band', 'cool...",Count Basie,67


## Additional Feature Engineering

I also want to get the genre data to put on tracks. The genre data only comes from the artist objects, so we'll take that data and put it on the tracks. Note: not all artists have genre data, so this won't be incredibly useful, but it'll be cool with what we have!

In [18]:
def get_genres_from_artists(entry):
    artists = ast.literal_eval(entry["artist_ids"])
    genres_set = set()
    for artist in artists:
        genres = artist_data_df[artist_data_df["id"] == artist].iloc[0]["genres"]
        for genre in genres:
            genres_set.add(genre)
    return list(genres_set)

if not "genres" in full_track_data_df:
    full_track_data_df["genres"] = full_track_data_df.apply(get_genres_from_artists, axis=1)
    full_track_data_df.to_csv("data/ready/FullTrackData.csv")
full_track_data_df.head()

,id,album_id,album_name,album_release_date,album_total_tracks,artist_ids,artist_names,duration_ms,explicit,name,popularity,genres
0,4hns23kYYZg0BhDwXeDxB1,0Gwu5X7W1mrkSTk2uZ25cv,Blues & Politics,1999-06-29,8,['54YNxT02JdAApvFBhD8ea0'],['Mingus Big Band'],499826,False,Haitian Fight Song,30,"['bebop', 'big band', 'modern big band', 'jazz']"
1,3sLmks6fCY40bBSGDjU4FO,7By1lfK4fTIs2YsMvA0FWH,The Project,2004-01-01,10,['5zS6TsJ4lQFUGePSHAXaI9'],['Andy Martin & Vic Lewis'],459106,False,Everything You Is,0,[]
2,2408a07TNDga6lMlaIFLEU,5gfrrR8BnDgFhqGWcQaWFe,And the Danish Radio Big Band & Eclipse,2013-05-10,22,['6DbqS0X8cSFOPGsvyze2yh'],['Thad Jones'],255466,False,To You,7,"['hard bop', 'jazz trumpet', 'soul jazz', 'big..."
3,4Saza06xljloZwotqXdNle,3f22Ap0VSZYWsqrGcphUnY,Christmas Classics,2004-01-01,16,['5fJ4w85NxFXyWlPU9wH6BE'],['Guy Lombardo & His Royal Canadians'],128000,False,Auld Lang Syne,34,[]
4,0rupt7DuLo3WGecL3cyi19,2kAN1sZjSQQDkusyXyngep,Frankly Basie / Count Basie Plays The Hits Of ...,1963-01-01,15,['2jFZlvIea42ZvcCw4OeEdA'],['Count Basie'],150773,False,All Of Me,15,"['stride', 'lounge', 'swing', 'vocal jazz', 'b..."


I also want to figure out approximately how many times I've listened to a track. I kinda bounced a few ideas around on how to calculate this, but I decided that the best way to calculate this is to add up all of the milliseconds I've played the songs (`ms_played`) and determine how many total times I've played a song by dividing that number by the length of the song and simply rounding the result:

In [19]:
old_df = pd.read_csv("data/ready/EnhancedTracksData.csv")
old_df.drop(old_df.columns[0], axis=1, inplace=True)
old_df.head()

,id,date,ms_played,artist_ids,duration_ms,total_ms_played,total_times_played,time
0,4hns23kYYZg0BhDwXeDxB1,2020-01-01,680,['54YNxT02JdAApvFBhD8ea0'],499826,5877,0,00:51:00
1,3sLmks6fCY40bBSGDjU4FO,2020-01-01,3282,['5zS6TsJ4lQFUGePSHAXaI9'],459106,466281,1,00:52:00
2,4Saza06xljloZwotqXdNle,2020-01-01,66171,['5fJ4w85NxFXyWlPU9wH6BE'],128000,66171,1,00:52:00
3,2408a07TNDga6lMlaIFLEU,2020-01-01,1496,['6DbqS0X8cSFOPGsvyze2yh'],255466,273923,1,00:52:00
4,0rupt7DuLo3WGecL3cyi19,2020-01-01,150773,['2jFZlvIea42ZvcCw4OeEdA'],150773,452199,3,00:55:00


In [20]:
if not path.exists("data/ready/EnhancedTracksData.csv"):
    enhanced_tracks_data_df = combined_tracks_data_df.merge(full_track_data_df[["id", "artist_ids", "duration_ms"]], on="id")
    total_ms_played_df = enhanced_tracks_data_df[["id", "ms_played"]].groupby("id", as_index=False)["ms_played"].sum().rename({"ms_played": "total_ms_played"}, axis=1)
    enhanced_tracks_data_df = enhanced_tracks_data_df.merge(total_ms_played_df, on="id")
    enhanced_tracks_data_df["total_times_played"] = enhanced_tracks_data_df.apply(lambda entry: round(entry["total_ms_played"] / entry["duration_ms"]), axis=1)

    def split_date(row):
        date = pd.to_datetime(row["date"])
        return pd.Series([date.date(), date.time()])
    enhanced_tracks_data_df[["date", "time"]] = enhanced_tracks_data_df.apply(split_date, axis=1)

    enhanced_tracks_data_df.drop(columns=["album_id"], inplace=True)
    enhanced_tracks_data_df = enhanced_tracks_data_df.sort_values(by=["date", "time"]).reset_index(drop=True)
    enhanced_tracks_data_df.to_csv("data/ready/EnhancedTracksData.csv")
else:
    enhanced_tracks_data_df = pd.read_csv("data/ready/EnhancedTracksData.csv")
    enhanced_tracks_data_df.drop(enhanced_tracks_data_df.columns[0], axis=1, inplace=True)

enhanced_tracks_data_df.head()

,id,date,ms_played,artist_ids,duration_ms,total_ms_played,total_times_played,time
0,4hns23kYYZg0BhDwXeDxB1,2020-01-01,680,['54YNxT02JdAApvFBhD8ea0'],499826,5877,0,00:51:00
1,3sLmks6fCY40bBSGDjU4FO,2020-01-01,3282,['5zS6TsJ4lQFUGePSHAXaI9'],459106,466281,1,00:52:00
2,4Saza06xljloZwotqXdNle,2020-01-01,66171,['5fJ4w85NxFXyWlPU9wH6BE'],128000,66171,1,00:52:00
3,2408a07TNDga6lMlaIFLEU,2020-01-01,1496,['6DbqS0X8cSFOPGsvyze2yh'],255466,273923,1,00:52:00
4,0rupt7DuLo3WGecL3cyi19,2020-01-01,150773,['2jFZlvIea42ZvcCw4OeEdA'],150773,452199,3,00:55:00


So I found [this article](https://towardsdatascience.com/is-my-spotify-music-boring-an-analysis-involving-music-data-and-machine-learning-47550ae931de) that calculates the boringness of a song through this arbitrary equation:
```
boringness = loudness + tempo + (energy*100) + (danceability*100)
```
And I really want to find out how boring my music is, so...

In [21]:
def get_boringness_score(entry):
    return entry["loudness"] + entry["tempo"] + (entry["energy"] * 100) + (entry["danceability"] * 100)

if not "boringness" in track_feature_data_df:
    track_feature_data_df["boringness"] = track_feature_data_df.apply(get_boringness_score, axis=1)
    track_feature_data_df.to_csv("data/ready/TrackFeatureData.csv")
track_feature_data_df.head()

,id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,boringness
0,4hns23kYYZg0BhDwXeDxB1,0.386,0.392,2,-14.123,0,0.0408,0.536,0.794,0.1670,0.3990,89.153,4,152.830
1,3sLmks6fCY40bBSGDjU4FO,0.581,0.517,5,-10.203,0,0.0378,0.901,0.794,0.1080,0.5530,138.751,4,238.348
2,2408a07TNDga6lMlaIFLEU,0.138,0.110,5,-15.402,1,0.0337,0.945,0.943,0.1110,0.0534,61.730,4,71.128
3,4Saza06xljloZwotqXdNle,0.240,0.245,5,-10.777,1,0.0327,0.979,0.788,0.0947,0.1120,83.922,3,121.645
4,0rupt7DuLo3WGecL3cyi19,0.529,0.176,7,-17.229,1,0.0398,0.762,0.720,0.0529,0.7910,150.108,4,203.379


Okay, okay. I think it's finally time...

Time... for the analysis.